<a href="https://colab.research.google.com/github/zeligism/ConGAN/blob/main/ConsistentGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount drive

In [126]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Header

## Imports

In [127]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import glob
import random
import datetime
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision
import torchvision.utils as vutils
import torchvision.transforms as transforms
import torch.utils.tensorboard as tensorboard

from PIL import Image, ImageDraw
from math import log2
from pprint import pformat
from collections import defaultdict

## Utility Functions

### Report Utils

In [128]:
def plot_lines(losses_dict, filename=None, title=""):
    """
    Plots the losses of the discriminator and the generator.

    Args:
        filename: The plot's filename. If None, plot won't be saved.
    """

    plt.figure(figsize=(10,5))
    plt.title(title)
    for label, losses in losses_dict.items():
        plt.plot(losses, label=label)
    plt.xlabel("t")
    plt.legend()
    
    if filename is not None:
        plt.savefig(filename)
    
    plt.show()
    plt.close()


def create_progress_animation(frames, filename):
    """
    Creates a video of the progress of the generator on a fixed latent vector.

    Args:
        filename: The animation's filename.
    """

    fig = plt.figure(figsize=(8,8))
    plt.axis("off")
    ims = [[plt.imshow(img.permute(1,2,0), animated=True)]
           for img in frames]
    ani = animation.ArtistAnimation(fig, ims, blit=True)
    
    ani.save(filename)

    plt.close()


def generate_grid(generator, latent):
    """
    Check generator's output on latent vectors and return it.

    Args:
        generator: The generator.
        latent: Latent vector from which an image grid will be generated.

    Returns:
        A grid of images generated by `generator` from `latent`.
    """

    with torch.no_grad():
        fake = generator(latent).detach()

    image_grid = vutils.make_grid(fake.cpu(), padding=2, normalize=True, range=(-1,1))

    return image_grid



# Classes

## PyTorch Modules

### DCGAN

In [309]:
class ConvBlock(nn.Module):
    """
    Default stride and padding half the size of features,
    e.g. if input is [in_channels, 64, 64], output will be [out_channels, 32, 32].
    """
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1,
                 use_batchnorm=True, use_spectralnorm=False, activation=None):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding, bias=False)
        if use_spectralnorm:
            self.conv = nn.utils.spectral_norm(self.conv)
        self.batchnorm = nn.BatchNorm2d(out_channels) if use_batchnorm else None
        self.activation = nn.LeakyReLU(0.2, inplace=True) if activation is None else activation()

    def forward(self, x):
        x = self.conv(x)
        if self.batchnorm:
            x = self.batchnorm(x)
        x = self.activation(x)
        return x


class ConvTBlock(nn.Module):
    """
    Default stride and padding double the size of features,
    e.g. if input is [in_channels, 32, 32], output will be [out_channels, 64, 64].
    """
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1,
                 use_batchnorm=True, use_spectralnorm=False, activation=None):
        super().__init__()

        self.convT = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)
        if use_spectralnorm:
            self.convT = nn.utils.spectral_norm(self.convT)
        self.batchnorm = nn.BatchNorm2d(out_channels) if use_batchnorm else None
        self.activation = nn.ReLU(inplace=True) if activation is None else activation()

    def forward(self, x):
        x = self.convT(x)
        if self.batchnorm:
            x = self.batchnorm(x)
        x = self.activation(x)
        return x


class DCGAN_Discriminator(nn.Module):
    def __init__(self,
                 num_latents=16,
                 num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 max_features=512,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,
                 output_sigmoid=True,
                 D_block=ConvBlock):
        super().__init__()

        using_grad_penalty = gan_type in ("gan-gp", "wgan-gp")
        output_sigmoid = output_sigmoid and gan_type in ("gan", "gan-gp")

        block_config = {
            "activation": activation,
            "use_batchnorm": use_batchnorm and not using_grad_penalty,
            "use_spectralnorm": use_spectralnorm,
        }

        # Calculate intermediate image sizes
        image_sizes = [image_size]
        while image_sizes[-1] > 5:
            image_sizes.append(image_sizes[-1] // 2)
        latent_kernel = image_sizes[-1]  # should be either 3, 4, or 5
        num_layers = len(image_sizes) - 1

        # Calculate feature sizes
        features = [min(max_features, round(num_features * feature_multiplier**i))
                    for i in range(num_layers)]

        # Input layer
        self.input_layer = D_block(image_channels, features[0], **block_config)

        # Intermediate layers
        self.main_layers = nn.Sequential(*[
            D_block(in_features, out_features, **block_config)
            for in_features, out_features in zip(features, features[1:])
        ])

        # Output layer (feature_size = 3, 4, or 5 -> 1)
        if fully_convolutional:
            self.output_layer = nn.Sequential(
                nn.Conv2d(features[-1], num_latents, latent_kernel, bias=False),
                nn.Flatten(),
            )
        else:
            self.output_layer = nn.Sequential(
                nn.Flatten(),
                nn.Linear(features[-1] * latent_kernel**2, num_latents, bias=False)
            )

        # Add sigmoid activation if using regular GAN loss
        self.output_activation = nn.Sigmoid() if output_sigmoid else None

    def forward(self, x):
        x = self.input_layer(x)
        x = self.main_layers(x)
        x = self.output_layer(x)
        if self.output_activation:
            x = self.output_activation(x)
        # Remove H and W dimensions, infer channels dim (remove if 1)
        x = x.view(x.size(0), -1).squeeze(1)
        return x


class DCGAN_Generator(nn.Module):
    def __init__(self,
                 num_latents=100,
                 num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 max_features=512,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,
                 G_block=ConvTBlock):
        super().__init__()

        block_config = {
            "activation": activation,
            "use_batchnorm": use_batchnorm,
            "use_spectralnorm": use_spectralnorm
        }

        # Calculate intermediate image sizes
        image_sizes = [image_size]
        while image_sizes[-1] > 5:
            image_sizes.append(image_sizes[-1] // 2)
        latent_kernel = image_sizes[-1]  # should be either 3, 4, or 5
        num_layers = len(image_sizes) - 1

        # Calculate feature sizes
        features = [min(max_features, round(num_features * feature_multiplier**i))
                    for i in range(num_layers)]

        # Reverse order of image sizes and features for generator
        image_sizes = image_sizes[::-1]
        features = features[::-1]

        # Input layer
        if fully_convolutional:
            self.input_layer = G_block(num_latents, features[0], kernel_size=latent_kernel,
                                       stride=1, padding=0, **block_config)
        else:
            self.input_layer = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_latents, features[0] * image_sizes[0]**2, bias=False),
                View(features[0], image_sizes[0], image_sizes[0])
            )

        # Intermediate layers
        self.main_layers = nn.Sequential(*[
            G_block(in_features, out_features, kernel_size=4+(expected_size%2), **block_config)
            for in_features, out_features, expected_size in zip(features, features[1:], image_sizes[1:])
        ])

        # Output layer
        self.output_layer = nn.ConvTranspose2d(features[-1], image_channels, kernel_size=4+(image_size%2),
                                               stride=2, padding=1, bias=False)
        self.output_activation = nn.Tanh()

    def forward(self, x):
        # Add H and W dimensions, infer channels dim (add if none)
        x = x.view(x.size(0), -1, 1, 1)
        x = self.input_layer(x)
        x = self.main_layers(x)
        x = self.output_layer(x)
        x = self.output_activation(x)
        return x


class DCGAN(nn.Module):
    """Deep Convolutional Generative Adversarial Network"""

    def __init__(self,
                 num_latents=100,
                 D_num_features=64,
                 G_num_features=64,
                 image_channels=3,
                 image_size=64,
                 feature_multiplier=2,
                 gan_type="gan",
                 fully_convolutional=True,
                 activation=None,
                 use_batchnorm=True,
                 use_spectralnorm=False,
                 output_sigmoid=True,):
        """
        Initializes DCGAN.

        Args:
            num_latents: Number of latent factors.
            num_features: Number of features in the convolutions.
            image_channels: Number of channels in the input image.
            image_size: Size (i.e. height or width) of image.
            gan_type: Type of GAN (e.g. "gan" or "wgan-gp").
        """
        super().__init__()

        self.num_latents = num_latents
        self.D_num_features = D_num_features
        self.G_num_features = G_num_features
        self.image_channels = image_channels
        self.image_size = image_size
        self.feature_multiplier = feature_multiplier
        self.gan_type = gan_type
        self.fully_convolutional = fully_convolutional
        self.activation = activation
        self.use_batchnorm = use_batchnorm
        self.use_spectralnorm = use_spectralnorm

        D_params = {
            "num_latents": 1,  # XXX
            "num_features": D_num_features,
            "image_channels": image_channels,
            "image_size": image_size,
            "feature_multiplier": feature_multiplier,
            "gan_type": gan_type,
            "fully_convolutional": fully_convolutional,
            "activation": activation,
            "use_batchnorm": use_batchnorm,
            "use_spectralnorm": use_spectralnorm,
            "output_sigmoid": output_sigmoid,
        }
        G_params = {
            "num_latents": num_latents,
            "num_features": G_num_features,
            "image_channels": image_channels,
            "image_size": image_size,
            "feature_multiplier": feature_multiplier,
            "gan_type": gan_type,
            "fully_convolutional": fully_convolutional,
            "activation": activation,
            "use_batchnorm": True,
            "use_spectralnorm": False,  # XXX
        }

        self.D = DCGAN_Discriminator(**D_params)
        self.G = DCGAN_Generator(**G_params)


class View(nn.Module):
    def __init__(self, *shape, including_batch=False):
        super().__init__()
        self.shape = shape
        self.including_batch = including_batch
    
    def forward(self, x):
        if self.including_batch:
            return x.view(*self.shape)
        else:
            return x.view(x.size(0), *self.shape)

class ChannelNoise(nn.Module):
    """
    Channel noise injection module.
    Adds a linearly transformed noise to a convolution layer.
    """

    def __init__(self, num_channels, std=0.02):
        super().__init__()
        self.std = std
        self.scale = nn.Parameter(torch.ones(1, num_channels, 1, 1))


    def forward(self, x):
        noise_size = [x.size()[0], 1, *x.size()[2:]]  # single channel
        noise = self.std * torch.randn(noise_size).to(x)

        return x + self.scale * noise

### SimSiam

In [130]:
class SimSiam(nn.Module):
    """
    Build a SimSiam model.
    """
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        """
        dim: feature dimension (default: 2048)
        pred_dim: hidden dimension of the predictor (default: 512)
        """
        super(SimSiam, self).__init__()

        # create the encoder
        # num_classes is the output fc dimension, zero-initialize last BNs
        self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)

        # build a 3-layer projector
        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        # build a 2-layer predictor
        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        """
        Input:
            x1: first views of images
            x2: second views of images
        Output:
            p1, p2, z1, z2: predictors and targets of the network
            See Sec. 3 of https://arxiv.org/abs/2011.10566 for detailed notations
        """

        # compute features for one view
        z1 = self.encoder(x1) # NxC
        z2 = self.encoder(x2) # NxC

        p1 = self.predictor(z1) # NxC
        p2 = self.predictor(z2) # NxC

        return p1, p2, z1.detach(), z2.detach()

### ConsistentGAN

In [131]:
class ConsistentGAN(nn.Module):
    def __init__(self, repr_dim, latent_dim,
                 D_batchnorm=True, image_size=64, *args, **kwargs):
        super().__init__()
        self.repr_dim = repr_dim
        self.latent_dim = latent_dim
        
        # Make D's architecture kinda similar to predictor
        D_hidden_dim = repr_dim // 10
        if D_batchnorm:
            self.D = nn.Sequential(nn.Linear(repr_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                                   nn.BatchNorm1d(D_hidden_dim),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, 1),
                                   nn.Sigmoid())
        else:
            self.D = nn.Sequential(nn.Linear(repr_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, D_hidden_dim, bias=False),
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(D_hidden_dim, 1),
                                   nn.Sigmoid())


        # Same for generator (latent -> representations)
        G_hidden_dim = repr_dim // 10
        self.G = nn.Sequential(nn.Linear(latent_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, G_hidden_dim, bias=False),
                               nn.BatchNorm1d(G_hidden_dim),
                               nn.LeakyReLU(0.2, inplace=True),
                               nn.Linear(G_hidden_dim, repr_dim, bias=False),
                               nn.BatchNorm1d(repr_dim, affine=False),
                               Normalize2()
                               )

        # Encodes x to context @XXX
        ctx_dim = 256
        self.ctx_encoder = DCGAN_Discriminator(num_latents=ctx_dim, image_size=image_size)

        # Projections to decoding space
        # Decodes representation + context projection to an image
        num_latents = 512
        self.repr_proj = nn.Sequential(nn.Linear(repr_dim, num_latents // 2), nn.Sigmoid())
        self.ctx_proj = nn.Sequential(nn.Linear(ctx_dim, num_latents // 2), nn.Sigmoid())
        self.decoder = DCGAN_Generator(num_latents=num_latents, image_size=image_size)

        # To check progress of G
        self.fixed_latent = self.sample_latent(8*8)
    
    # XXX temp function
    def proj(self, repr, ctx, return_all=False):
        repr_proj = self.repr_proj(repr)
        ctx_proj = self.ctx_proj(ctx)
        proj = torch.cat([repr_proj, ctx_proj], dim=1)
        if return_all:
            return proj, repr_proj, ctx_proj
        else:
            return proj

    def sample_latent(self, batch_size):
        latent_size = [batch_size, self.latent_dim]
        latent = torch.randn(latent_size)
        return latent


class Normalize2(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return F.normalize(x, p=2, dim=1)


# Training v2

### Imports and globals

In [132]:
import argparse
import builtins
import math
import os
import random
import shutil
import time
import warnings
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

GANSIAM_DIR = "/content/drive/My Drive/gansiam/"
SIMSIAM_PATH = os.path.join(GANSIAM_DIR, "pretrained_batch256.tar")
TINYIMAGENET_DIR = "tiny-imagenet-200"

### Download Tiny Imagenet

In [133]:
%%bash
if [[ -d  "tiny-imagenet-200" ]]; then
    echo "Tiny Imagenet exists."
else
    wget -q "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    unzip -qq "tiny-imagenet-200.zip" && rm "tiny-imagenet-200.zip"
    echo "Downloaded Tiny Imagenet."
fi

Tiny Imagenet exists.


### Utility

#### GAN Utils

In [134]:
def get_D_loss(gan_type="gan"):
    if gan_type in ("gan", "gan-gp"):
        return D_loss_GAN
    elif gan_type in ("wgan", "wgan-gp"):
        return D_loss_WGAN
    else:
        raise ValueError(f"gan_type {gan_type} not supported")


def get_G_loss(gan_type="gan"):
    if gan_type in ("gan", "gan-gp"):
        return G_loss_GAN
    elif gan_type in ("wgan", "wgan-gp"):
        return G_loss_WGAN
    else:
        raise ValueError(f"gan_type {gan_type} not supported")


def D_loss_GAN(D_real, D_fake, label_smoothing=True):
    
    # Create (noisy) real and fake labels XXX
    if label_smoothing:
        real_label = 0.7 + 0.5 * torch.rand_like(D_real)
    else:
        real_label = torch.ones_like(D_real) - 0.1
    fake_label = torch.zeros_like(D_fake)

    # Calculate binary cross entropy loss
    D_loss_real = F.binary_cross_entropy(D_real, real_label)
    D_loss_fake = F.binary_cross_entropy(D_fake, fake_label)

    # Loss is: - log(D(x)) - log(1 - D(x_g)),
    # which is equiv. to maximizing: log(D(x)) + log(1 - D(x_g))
    D_loss = D_loss_real + D_loss_fake

    return D_loss.mean()


def D_loss_WGAN(D_real, D_fake):

    # Maximize: D(x) - D(x_g) - const * (|| grad of D(x_i) wrt x_i || - 1)^2,
    # where x_i <- eps * x + (1 - eps) * x_g, and eps ~ rand(0,1)
    D_loss = -1 * (D_real - D_fake)

    return D_loss.mean()


def G_loss_GAN(D_fake):

    # Calculate binary cross entropy loss with a fake binary label
    fake_label = torch.zeros_like(D_fake)

    # Loss is: -log(D(G(z))), which is equiv. to minimizing log(1-D(G(z)))
    # We use this loss vs. the original one for stability only.
    G_loss = F.binary_cross_entropy(D_fake, 1 - fake_label)

    return G_loss.mean()


def G_loss_WGAN(D_fake):

    # Minimize: -D(G(z))
    G_loss = -D_fake
    
    return G_loss.mean()


def interpolate(real, fake):
    eps_size = [1] * len(real.size())
    eps_size[0] = real.size(0)
    eps = torch.rand(eps_size).to(real)
    return eps * real + (1 - eps) * fake

def simple_gradient_penalty(D, x, center=0.):
    x.requires_grad_()
    D_x = D(x)
    D_grad = torch.autograd.grad(D_x, x, torch.ones_like(D_x), create_graph=True)
    D_grad_norm = D_grad[0].view(x.size(0), -1).norm(dim=1)
    return (D_grad_norm - center).pow(2).mean()


#### Data Utils

In [135]:
from PIL import ImageFilter
import random


class TwoCropsTransform:
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]


class GaussianBlur(object):
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[.1, 2.]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(ImageFilter.GaussianBlur(radius=sigma))
        return x

#### Train Utils

In [136]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    filename = os.path.join(GANSIAM_DIR, "results", filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def adjust_learning_rate(optimizer, init_lr, epoch, args):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / args.epochs))
    for param_group in optimizer.param_groups:
        if 'fix_lr' in param_group and param_group['fix_lr']:
            param_group['lr'] = init_lr
        else:
            param_group['lr'] = cur_lr


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


### Load pre-trained SimSiam model


In [137]:
checkpoint = torch.load(SIMSIAM_PATH, map_location="cuda:0")
# remove 'module.' from dict keys
model_dict = OrderedDict((k[7:], v) for k, v in checkpoint["state_dict"].items())

# Load model
simsiam = SimSiam(models.__dict__["resnet50"])
simsiam.load_state_dict(model_dict)
#print(simsiam)

<All keys matched successfully>

# Training v3

## Args

In [317]:
class Args:
    def __init__(self):
        self.data = TINYIMAGENET_DIR
        self.workers = 2
        self.epochs = 100
        self.batch_size = 256
        self.D_lr = 2e-4
        self.G_lr = 2e-4
        self.betas = (0., 0.9)
        #self.momentum = 0.9
        #self.weight_decay = 1e-4
        self.print_freq = 10
        self.seed = None
        self.gpu = 0

        # SimSiam (don't change if loading pre-trained)
        self.dim = 2048
        self.pred_dim = 512

        # GAN
        self.gan_type = "gan"
        self.repr_dim = self.dim  # don't change
        self.latent_dim = 128
        self.num_features = 64
        self.D_iters = 5
        self.grad_penalty = 0.
        self.grad_center = 1.
        self.generate_grid_interval = 100

        # noise is assumed to be proportional to 1e-3 * sd(data)
        self.im_noise = 1e-3  # image sd is about 1.0
        self.repr_noise = 1e-6  # repr sd is about 0.001
        self.repr_consistency = 0.1  # how much should G be consistent in repr XXX
        self.latent_transform_lr = self.G_lr


GENERATED_GRIDS = []
IMAGE_SIZE = 64
args = Args()

## Data

In [318]:
# image normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=mean, std=std)
inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(mean, std)],
   std= [1/s for s in std]
)


# MoCo v2's aug: similar to SimCLR https://arxiv.org/abs/2002.05709
"""
augmentation = [
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.2, 1.)),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]
"""

augmentation = [
    #transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]

DATASET = "CIFAR10"

if DATASET == "MNIST":
    augmentation = [transforms.Grayscale(3)] + augmentation
    train_dataset = datasets.MNIST(
        root=os.path.join(GANSIAM_DIR, "mnist/train"), train=True, download=True,
        transform=transforms.Compose(augmentation))
elif DATASET == "CelebA":
    train_dataset = datasets.CelebA(
        root=os.path.join(GANSIAM_DIR, "celeba"), download=True,
        transform=transforms.Compose(augmentation))
elif DATASET == "CIFAR10":
    train_dataset = datasets.CIFAR10(
        root=os.path.join(GANSIAM_DIR, "cifar10/train"), train=True, download=True,
        transform=transforms.Compose(augmentation))
        #transform=TwoCropsTransform(transforms.Compose(augmentation)))
elif DATASET == "Tiny Imagenet":
    train_dataset = datasets.ImageFolder(
        root=os.path.join(TINYIMAGENET_DIR, 'train'),
        transform=transforms.Compose(augmentation))
else:
    raise Exception(f"Dataset '{DATASET}' not found")

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True, sampler=None, drop_last=True)

Files already downloaded and verified


## Model + Opt

In [319]:
def D_criterion_NS(D_real, D_fake):
    d_loss = F.softplus(-D_real) + F.softplus(D_fake)
    return d_loss.mean()

def G_criterion_NS(D_fake):
    return F.softplus(-D_fake).mean()

def D_criterion_LS(D_real, D_fake):
    d_loss = 0.5 * (D_real - torch.ones_like(D_real))**2 + 0.5 * (D_fake)**2
    return d_loss.mean()

def G_criterion_LS(D_fake):
    gen_loss = 0.5 * (D_fake - torch.ones_like(D_fake))**2
    return gen_loss.mean()

def D_criterion_hinge(D_real, D_fake):
    return torch.mean(F.relu(1. - D_real)) + torch.mean(F.relu(1. + D_fake))

def G_criterion_hinge(D_fake):
    return -torch.mean(D_fake)

In [320]:

load = False

if args.seed is not None:
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    cudnn.deterministic = True

torch.cuda.set_device(args.gpu)

hidden_latent_dim = 2*args.latent_dim  # XXX
latent_transform = nn.Sequential(
    nn.Linear(args.repr_dim+args.latent_dim, hidden_latent_dim, bias=False),
    #nn.Linear(hidden_latent_dim, hidden_latent_dim, bias=False),
    #nn.Linear(hidden_latent_dim, hidden_latent_dim, bias=False),
    #nn.Linear(hidden_latent_dim, hidden_latent_dim, bias=False),
    #nn.Linear(hidden_latent_dim, hidden_latent_dim, bias=False),
)

model = DCGAN(num_latents=hidden_latent_dim,
              image_size=IMAGE_SIZE,
              gan_type=args.gan_type,
              use_batchnorm=False,
              use_spectralnorm=True,
              output_sigmoid=False,
              D_num_features=args.num_features,
              G_num_features=args.num_features*2,
              )

latent_transform = latent_transform.cuda(args.gpu)
model = model.cuda(args.gpu)
simsiam = simsiam.cuda(args.gpu)

# Define D and G loss functions
D_criterion = D_criterion_LS
G_criterion = G_criterion_LS

# Optimizers
D_optimizer = torch.optim.Adam(
    model.D.parameters(),
    args.D_lr, betas=args.betas)

G_optimizer = torch.optim.Adam(
    [{"params": model.G.parameters()},
     {"params": latent_transform.parameters(), "lr": args.latent_transform_lr}],
     args.G_lr, betas=args.betas)

D_sched = torch.optim.lr_scheduler.ExponentialLR(D_optimizer, 0.99)
G_sched = torch.optim.lr_scheduler.ExponentialLR(G_optimizer, 0.99)

cudnn.benchmark = True

print("Num of params in D:", sum(map(torch.numel, model.D.parameters())))
print("Num of params in G:", sum(map(torch.numel, model.G.parameters())))

if load:
    model.load_state_dict(torch.load(f"{GANSIAM_DIR}/results/model.pth.tar"))
    latent_transform.load_state_dict(torch.load(f"{GANSIAM_DIR}/results/latent_transform.pth.tar"))
    D_sched.load_state_dict(torch.load(f"{GANSIAM_DIR}/results/D_sched.pth.tar"))
    G_sched.load_state_dict(torch.load(f"{GANSIAM_DIR}/results/G_sched.pth.tar"))

Num of params in D: 2763776
Num of params in G: 8921856


## Train

In [321]:
def sample_latent(num_samples):
    return torch.randn(num_samples, args.latent_dim)

def check_G_progress(G):
    with torch.no_grad():
        fake_progress = G(latent_transform(fixed_latent))
    im_grid = torch.cat([fixed_x, fake_progress], dim=0)
    grid = vutils.make_grid(im_grid.cpu(), padding=2, normalize=True, range=(-1,1))
    return grid

In [322]:
# Sample a global latent for reuse
fixed_x, _ = next(iter(train_loader))
fixed_x = fixed_x[:32].cuda(args.gpu)
with torch.no_grad():
    fixed_repr = simsiam.encoder(fixed_x)
    fixed_repr = F.normalize(fixed_repr)
fixed_noise = sample_latent(32).cuda(args.gpu)
fixed_latent = torch.cat([fixed_repr, fixed_noise], dim=1)

In [323]:
def train(train_loader, model, simsiam,
          D_criterion, G_criterion, D_optimizer, G_optimizer, epoch, args):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    D_on_reals = AverageMeter('D(real)', ':.4f')
    D_on_fakes = AverageMeter('D(fake)', ':.4f')
    D_grads = AverageMeter('grad(D)', ':.4f')
    repr_losses = AverageMeter('repr loss', ':.4f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time,
         D_on_reals, D_on_fakes, D_grads, repr_losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()
    D_grad_penalty = torch.zeros(1).cuda(args.gpu)
    repr_loss = torch.zeros(1).cuda(args.gpu)

    end = time.time()
    for i, (x, y) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        x = x.cuda(args.gpu, non_blocking=True)
        batch_size = x.size(0)

        # compute output and loss
        with torch.no_grad():
            repr = simsiam.encoder(x)
            repr = F.normalize(repr + args.repr_noise * torch.randn_like(repr))

        ### train GAN
        #if (i+1) % (args.D_iters+1) > 0:
        for _ in range(args.D_iters):
            # Add noise to real sample
            real = x + args.im_noise * torch.randn_like(x)

            # Sample from generator
            with torch.no_grad():
                # Sample latent
                latent_noise = sample_latent(batch_size).cuda(args.gpu)
                latent = torch.cat([repr, latent_noise], dim=1)
                fake = model.G(latent_transform(latent))
                # Add noise to fake sample as well
                fake = fake + args.im_noise * torch.randn_like(fake)

            # Classify real and fake data
            D_real = model.D(real)
            D_fake = model.D(fake)

            # Calculate loss
            D_loss = D_criterion(D_real, D_fake)
            # Gradient penalty
            if args.grad_penalty != 0.:
                D_grad_penalty = simple_gradient_penalty(
                    model.D, interpolate(real, fake), center=args.grad_center)
                D_loss += args.grad_penalty * D_grad_penalty

            # Calculate gradient and minimize
            D_optimizer.zero_grad()
            D_loss.backward()
            D_optimizer.step()

            # Save data
            D_on_reals.update(D_real.mean().item(), batch_size)
            D_on_fakes.update(D_fake.mean().item(), batch_size)
            D_grads.update(D_grad_penalty.mean().item(), batch_size)

        else:
            # Sample from generator
            latent_noise = sample_latent(batch_size).cuda(args.gpu)
            latent = torch.cat([repr, latent_noise], dim=1)
            fake = model.G(latent_transform(latent))
            fake = fake + args.im_noise * torch.randn_like(fake)
            # Classify fake images
            D_fake = model.D(fake)
            # Calculate loss
            G_loss = G_criterion(D_fake)
            if args.repr_consistency != 0.:
                # Representation consistency loss
                fake_repr = simsiam.encoder(fake)
                repr_loss = -F.cosine_similarity(fake_repr, repr).mean()
            # Calculate gradient and minimize
            G_optimizer.zero_grad()
            (G_loss + args.repr_consistency * repr_loss).backward()
            G_optimizer.step()

            # Save data
            D_on_fakes.update(D_fake.mean().item(), batch_size)
            repr_losses.update(repr_loss.mean().item(), batch_size)

        # Check generator's progress by recording its output on a fixed input
        if i % args.generate_grid_interval == 0:
            grid = check_G_progress(model.G)
            GENERATED_GRIDS.append(grid)

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


## Run

In [ ]:
for epoch in range(args.epochs):

    if epoch == 0:
        args.repr_consistency = 0.1
    if epoch == 100:
        args.repr_consistency = 10.

    # train for one epoch
    train(train_loader, model, simsiam,
          D_criterion, G_criterion, D_optimizer, G_optimizer, epoch, args)

    D_sched.step()
    G_sched.step()
    
    # Check G's progress evey epoch by generating an image
    grid = check_G_progress(model.G)
    imname = f'{GANSIAM_DIR}/results/progress/grid_{epoch:04d}.png'
    plt.imsave(imname, grid.permute(1,2,0).numpy())

    if epoch % 10 == 0:
        torch.save({'state_dict': model.state_dict()},
                f"{GANSIAM_DIR}/results/model.pth.tar")
        torch.save({'state_dict': latent_transform.state_dict()},
                f"{GANSIAM_DIR}/results/latent_transform.pth.tar")
        torch.save({'state_dict': D_sched.state_dict()},
                f"{GANSIAM_DIR}/results/D_sched.pth.tar")
        torch.save({'state_dict': G_sched.state_dict()},
                f"{GANSIAM_DIR}/results/G_sched.pth.tar")

/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)


Epoch: [0][  0/195]	Time  1.406 ( 1.406)	Data  0.342 ( 0.342)	D(real) 0.6454 (0.5813)	D(fake) 0.3982 (0.1623)	grad(D) 0.0000 (0.0000)	repr loss 0.0175 (0.0175)
Epoch: [0][ 10/195]	Time  0.982 ( 1.023)	Data  0.000 ( 0.031)	D(real) 0.3250 (0.8035)	D(fake) -0.0086 (0.0378)	grad(D) 0.0000 (0.0000)	repr loss -0.2935 (-0.1188)
Epoch: [0][ 20/195]	Time  0.995 ( 1.008)	Data  0.000 ( 0.017)	D(real) 0.8023 (0.8356)	D(fake) 0.0093 (0.0212)	grad(D) 0.0000 (0.0000)	repr loss -0.6414 (-0.3316)
Epoch: [0][ 30/195]	Time  0.976 ( 1.002)	Data  0.000 ( 0.011)	D(real) 0.7366 (0.8570)	D(fake) 0.0141 (0.0162)	grad(D) 0.0000 (0.0000)	repr loss -0.6216 (-0.4236)
Epoch: [0][ 40/195]	Time  0.989 ( 0.998)	Data  0.000 ( 0.009)	D(real) 0.7616 (0.8719)	D(fake) 0.0169 (0.0133)	grad(D) 0.0000 (0.0000)	repr loss -0.6901 (-0.4791)
Epoch: [0][ 50/195]	Time  0.997 ( 0.997)	Data  0.000 ( 0.007)	D(real) 1.0765 (0.8820)	D(fake) -0.0028 (0.0141)	grad(D) 0.0000 (0.0000)	repr loss -0.6724 (-0.5150)
Epoch: [0][ 60/195]	Time  0.

## Results

In [ ]:
%matplotlib inline

def show_sample(x, num_samples=16, show_x=False):
    x = x.cuda(args.gpu)[:num_samples]
    if show_x:
        x_grid = vutils.make_grid(inv_normalize(x).cpu(), padding=2, nrow=4)
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.imshow(x_grid.permute(1,2,0))

    model.eval()
    with torch.no_grad():
        latent_noise = sample_latent(num_samples).cuda(args.gpu)
        repr = simsiam.encoder(x)
        repr = F.normalize(repr)
        z = torch.cat([repr, latent_noise], dim=1)
        x_fake = model.G(latent_transform(z))
    im_grid = vutils.make_grid(x_fake.cpu(), padding=2, nrow=4, normalize=True, range=(-1,1))
    plt.figure(figsize=(8,8))
    plt.axis("off")
    plt.imshow(im_grid.permute(1,2,0))

x, _ = next(iter(train_loader))
show_sample(x, show_x=True)
show_sample(x)
show_sample(x)
show_sample(x)

In [ ]:
vidname = f"results/progress/grids_per_{args.generate_grid_interval}_iter.mp4"
vidname = os.path.join(GANSIAM_DIR, vidname)
create_progress_animation(GENERATED_GRIDS, vidname)